In [1]:
# mysql connection
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("tag_ods") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "1g") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/21 09:20:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/21 09:20:46 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/21 09:20:46 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [3]:
from datetime import datetime
date_string = datetime.today().strftime('%Y-%m-%d')
print(date_string)

2023-04-21


In [5]:
# get all user from db, and create an rdd
cur = mydb.cursor()
cur.execute(f'''
SELECT 
    tag.*,
    tag_category.*,
    '{date_string}' as date 
from 
    tag 
left join 
    tag_category on tag.category_id = tag_category.tag_category_id
''')
print(cur.column_names)
result = cur.fetchall()
all_tags = sc.parallelize(result)
print(all_tags.count())

('tag_id', 'tag_name', 'weight', 'mark_object', 'category_id', 'tag_category_id', 'tag_category_name', 'parent_id', 'level', 'date')


153


In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DecimalType
schema = StructType([
    StructField("tag_id", IntegerType(), True),
    StructField("tag_name", StringType(), True),
    StructField("tag_weight", DecimalType(8, 4), True),
    StructField("mark_object", StringType(), True),
    StructField("category_id", IntegerType(), True),
    StructField("tag_category_id", IntegerType(), True),
    StructField("category_name", StringType(), True),
    StructField("category_parent_id", IntegerType(), True),
    StructField("level", IntegerType(), True),
    StructField("date", StringType(), True)
])
tag_df = all_tags.toDF(schema)

In [7]:
spark.sql("drop table if exists tag_ods").show()

++
||
++
++



In [8]:
tag_df.write.mode("overwrite").saveAsTable("tag_ods")

23/04/21 09:22:11 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [9]:
spark.sql("show tables").show()
spark.sql("select * from tag_ods").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default| item_ods|      false|
| default|  tag_ods|      false|
| default|     test|      false|
| default|    test2|      false|
| default| user_ods|      false|
+--------+---------+-----------+

+------+----------+----------+-----------+-----------+---------------+-------------+------------------+-----+----------+
|tag_id|  tag_name|tag_weight|mark_object|category_id|tag_category_id|category_name|category_parent_id|level|      date|
+------+----------+----------+-----------+-----------+---------------+-------------+------------------+-----+----------+
|    77|  兽医服务|    1.0000|    service|          2|              2|    测试标签2|                 0|    1|2023-04-21|
|    78|农机具租赁|    1.0000|    service|          2|              2|    测试标签2|                 0|    1|2023-04-21|
|    79|  景点旅游|    1.0000|       live|          2|              2|    测试标签2|                 0|    1|2023-04-21|
|

In [10]:
spark.stop()
mydb.close()